In [1]:
from recbole.model.sequential_recommender import BERT4Rec 
import pandas as pd
from recbole.evaluator.metrics import RMSE , MAE , Hit , NDCG , Recall , MRR
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset , data_preparation
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
import numpy as np
import torch
from collections import defaultdict
from recbole.evaluator.metrics import Hit , NDCG , Recall
from recbole.quick_start import run_recbole , load_data_and_model

In [ ]:
df = pd.read_csv(r"C:\Users\Abdo\Desktop\both3.csv")
df = df.sort_values(by=['timestamp'])

In [ ]:
df1 = df.rename(columns={'userID': 'user_id:token', 'itemID': 'item_id:token', 'rating': 'rating:token' ,'timestamp': 'timestamp:float'})

In [ ]:
df1.to_csv(r'C:\Users\Abdo\Desktop\recbox_data\recbox_data.inter', index=False, sep='\t')

In [2]:
parameter_dict = {
    'data_path': r'C:\Users\Abdo\Desktop\recbox_data',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[ 0,100000]",
    'item_inter_num_interval': "[ 0,100000]",
    #'additional_feat_suffix': ['item'],
    'load_col': {'inter': ['user_id', 'item_id' , 'rating', 'timestamp'],
                 #'item': ['item_id', 'item_emb']
                },
    
    #'alias_of_item_id': ['item_id'],
    #'preload_weight':{'item_id': 'item_emb'},
    
    'unused_col':{'inter': ['rating'],
                  #'item': ['item_id', 'item_emb']
                 },
    'metrics': ['Recall','NDCG','Hit'],
    'topk'   : [1,5,10,20],
    'train_neg_sample_args':None,
    'seed':42,
    'train_batch_size': 1024,
    'eval_step': 1 ,                    # (int) The number of training epochs before an evaluation on the valid dataset.
    'n_layers': 2 ,                     # (int) The number of transformer layers in transformer encoder.
    'n_heads': 2  ,                    # (int) The number of attention heads for multi-head attention layer.
    'hidden_size': 64 ,                 # (int) The number of features in the hidden state.
    'inner_size': 256  ,               # (int) The inner hidden size in feed-forward layer.
    'hidden_dropout_prob': 0.2 ,        # (float) The probability of an element to be zeroed.
    'attn_dropout_prob': 0.2 ,          # (float) The probability of an attention score to be zeroed.
    'hidden_act': 'gelu' ,             # (str) The activation function in feed-forward layer.
    'layer_norm_eps': 1e-12 ,           # (float) A value added to the denominator for numerical stability.
    'initializer_range': 0.02 ,         # (float) The standard deviation for normal initialization.
    'mask_ratio': 0.2 ,                 # (float) The probability for a item replaced by MASK token.
    'loss_type': 'CE' ,                # (str) The type of loss function.
    'ft_ratio': 0.5   ,               # (float) The probability of generating fine-tuning samples
    'neg_sampling': None,
    'epochs': 1,
    'eval_args': {
        'split': {'RS': [1,0,0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'uni100'
    }
}

config = Config(model= 'BERT4Rec', dataset='recbox_data_old', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)


In [3]:
dataset = create_dataset(config )
#logger.info(dataset)

In [4]:
train_data, valid_data, test_data = data_preparation(config, dataset)

18 Mar 21:45    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
18 Mar 21:45    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}]


In [5]:
model = BERT4Rec(config, train_data.dataset).to(config['device'])

In [6]:
trainer = Trainer(config, model)
# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

18 Mar 21:46    INFO  epoch 0 training [time: 72.44s, train loss: 1014.2681]
epoch 0 training [time: 72.44s, train loss: 1014.2681]
18 Mar 21:46    INFO  Saving current: saved\BERT4Rec.pth
Saving current: saved\BERT4Rec.pth


In [7]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file=r'C:\Users\Abdo\Desktop\recbox_data\saved\BERT4Rec.pth')

18 Mar 21:46    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = C:\Users\Abdo\Desktop\recbox_data\recbox_data_old
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}
repeatable = True
metrics = ['Recall', 'NDCG', 'Hit']
topk = [1, 5, 10, 20]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_se

# After training on old data

In [8]:
model.update_embedding(x=3)

In [ ]:
df = pd.read_csv(r"C:\Users\Abdo\Desktop\recbox_data\recbox_data_new\recbox_data_new.csv")
df = df.sort_values(by=['timestamp'])

In [ ]:
df1 = df.rename(columns={'userID': 'user_id:token', 'itemID': 'item_id:token', 'rating': 'rating:token' ,'timestamp': 'timestamp:float'})

In [ ]:
df1.to_csv(r'C:\Users\Abdo\Desktop\recbox_data\recbox_data_new\recbox_data_new.inter', index=False, sep='\t')

In [9]:
parameter_dict = {
    'data_path': r'C:\Users\Abdo\Desktop\recbox_data',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[ 0,100000]",
    'item_inter_num_interval': "[ 0,100000]",
    #'additional_feat_suffix': ['item'],
    'load_col': {'inter': ['user_id', 'item_id' , 'rating', 'timestamp'],
                 #'item': ['item_id', 'item_emb']
                },
    
    #'alias_of_item_id': ['item_id'],
    #'preload_weight':{'item_id': 'item_emb'},
    
    'unused_col':{'inter': ['rating'],
                  #'item': ['item_id', 'item_emb']
                 },
    'metrics': ['Recall','NDCG','Hit'],
    'topk'   : [1,5,10,20],
    'train_neg_sample_args':None,
    'seed':42,
    'train_batch_size': 1024 ,#10000000
    'eval_step': 1 ,                    # (int) The number of training epochs before an evaluation on the valid dataset.
    'n_layers': 2 ,                     # (int) The number of transformer layers in transformer encoder.
    'n_heads': 2  ,                    # (int) The number of attention heads for multi-head attention layer.
    'hidden_size': 64 ,                 # (int) The number of features in the hidden state.
    'inner_size': 256  ,               # (int) The inner hidden size in feed-forward layer.
    'hidden_dropout_prob': 0.2 ,        # (float) The probability of an element to be zeroed.
    'attn_dropout_prob': 0.2 ,          # (float) The probability of an attention score to be zeroed.
    'hidden_act': 'gelu' ,             # (str) The activation function in feed-forward layer.
    'layer_norm_eps': 1e-12 ,           # (float) A value added to the denominator for numerical stability.
    'initializer_range': 0.02 ,         # (float) The standard deviation for normal initialization.
    'mask_ratio': 0.2 ,                 # (float) The probability for a item replaced by MASK token.
    'loss_type': 'CE' ,                # (str) The type of loss function.
    'ft_ratio': 0.5   ,               # (float) The probability of generating fine-tuning samples
    'neg_sampling': None,
    'epochs': 1,
    'eval_args': {
        'split': {'RS': [1,0,0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'uni100'
    }
}

config = Config(model= 'BERT4Rec', dataset='recbox_data_new', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)


In [10]:
dataset = create_dataset(config )

In [11]:
train_data, valid_data, test_data = data_preparation(config, dataset)

18 Mar 21:46    INFO  [Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [1024] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
18 Mar 21:46    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}]


In [12]:
parameter_dict = {
    'data_path': r'C:\Users\Abdo\Desktop\recbox_data',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[ 0,100000]",
    'item_inter_num_interval': "[ 0,100000]",
    #'additional_feat_suffix': ['item'],
    'load_col': {'inter': ['user_id', 'item_id' , 'rating', 'timestamp'],
                 #'item': ['item_id', 'item_emb']
                },
    
    #'alias_of_item_id': ['item_id'],
    #'preload_weight':{'item_id': 'item_emb'},
    
    'unused_col':{'inter': ['rating'],
                  #'item': ['item_id', 'item_emb']
                 },
    'metrics': ['Recall','NDCG','Hit'],
    'topk'   : [1,5,10,20],
    'train_neg_sample_args':None,
    'seed':42,
    'train_batch_size': 1024,
    'eval_step': 1 ,                    # (int) The number of training epochs before an evaluation on the valid dataset.
    'n_layers': 2 ,                     # (int) The number of transformer layers in transformer encoder.
    'n_heads': 2  ,                    # (int) The number of attention heads for multi-head attention layer.
    'hidden_size': 64 ,                 # (int) The number of features in the hidden state.
    'inner_size': 256  ,               # (int) The inner hidden size in feed-forward layer.
    'hidden_dropout_prob': 0.2 ,        # (float) The probability of an element to be zeroed.
    'attn_dropout_prob': 0.2 ,          # (float) The probability of an attention score to be zeroed.
    'hidden_act': 'gelu' ,             # (str) The activation function in feed-forward layer.
    'layer_norm_eps': 1e-12 ,           # (float) A value added to the denominator for numerical stability.
    'initializer_range': 0.02 ,         # (float) The standard deviation for normal initialization.
    'mask_ratio': 0.2 ,                 # (float) The probability for a item replaced by MASK token.
    'loss_type': 'CE' ,                # (str) The type of loss function.
    'ft_ratio': 0.5   ,               # (float) The probability of generating fine-tuning samples
    'neg_sampling': None,
    'epochs': 1,
    'eval_args': {
        'split': {'RS': [1,0,0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'uni100'
    }
}

config1 = Config(model= 'BERT4Rec', dataset='recbox_data_old&new', config_dict=parameter_dict)

# init random seed
init_seed(config1['seed'], config1['reproducibility'])

init_logger(config1)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

In [13]:
trainer = Trainer(config1, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

18 Mar 21:49    INFO  epoch 0 training [time: 0.39s, train loss: 8.7413]
epoch 0 training [time: 0.39s, train loss: 8.7413]
epoch 0 training [time: 0.39s, train loss: 8.7413]
epoch 0 training [time: 0.39s, train loss: 8.7413]
18 Mar 21:49    INFO  Saving current: saved\BERT4Rec.pth
Saving current: saved\BERT4Rec.pth
Saving current: saved\BERT4Rec.pth
Saving current: saved\BERT4Rec.pth


In [14]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file=r'C:\Users\Abdo\Desktop\recbox_data\saved\BERT4Rec.pth')

18 Mar 21:49    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = C:\Users\Abdo\Desktop\recbox_data\recbox_data_old&new
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [1, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'uni100'}
repeatable = True
metrics = ['Recall', 'NDCG', 'Hit']
topk = [1, 5, 10, 20]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
fiel

18 Mar 21:49    INFO  recbox_data_old&new
The number of users: 12712
Average actions of users: 11.863504051608842
The number of items: 14871
Average actions of items: 10.141022192333557
The number of inters: 150797
The sparsity of the dataset: 99.92023017416956%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
recbox_data_old&new
The number of users: 12712
Average actions of users: 11.863504051608842
The number of items: 14871
Average actions of items: 10.141022192333557
The number of inters: 150797
The sparsity of the dataset: 99.92023017416956%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
recbox_data_old&new
The number of users: 12712
Average actions of users: 11.863504051608842
The number of items: 14871
Average actions of items: 10.141022192333557
The number of inters: 150797
The sparsity of the dataset: 99.92023017416956%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
recbox_data_old&new
The number of users: 12712
Average actions of users:

# prediction part

In [20]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 11)# 11 to cancel the first one 

In [21]:
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [22]:
len(external_user_ids)

12711

In [23]:
topk_items = defaultdict(list)
for external_user_id in external_user_ids:
    _, topk_iid_list = predict_for_all_item(external_user_id, dataset, model)
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items[external_user_id].append(external_item_list)
print(len(topk_items))

12711


In [28]:
topk_items['A1RPTVW5VEOSI'][0][1:]# I removed the first element because it's one of the interactions of the user

['B00028AZ6E',
 'B005DYJM70',
 'B001ASMGNU',
 'B01AZ37ANG',
 'B000W9BA1W',
 'B00NMZO092',
 'B00AYULZEQ',
 'B000X9EQ7Q',
 'B00UWHG76M',
 'B01AJJ09UW']